In [1]:
from firebird import chip
from firebird import aardvark as av
from test import fixtures as fx

import numpy as np

from pyspark import SparkContext, SparkConf
from datetime import datetime

In [2]:
#export PYSPARK_DRIVER_PYTHON=python3
#export PYSPARK_PYTHON=python3
#LPW_MESOS_MASTER              = "mesos://192.168.32.3:5050"
LPW_MESOS_MASTER              = "mesos://losrlmnp01.cr.usgs.gov:5050"
LPW_EXECUTOR_IMAGE            = "usgseros/lcmap-pyccd-worker:spark"
LPW_EXECUTOR_CORES            = 3
LPW_EXECUTOR_FORCE_PULL_IMAGE = "true"
LPW_SPARK_PARALLELIZATION     = 2

In [3]:
bands = ('blue', 'cfmask', 'green', 'nir', 'red', 'swir1', 'swir2', 'thermal')
g = globals()

In [4]:
# retrieve chips and specs
for band in bands:
    # chip specs per band... pertinent data should be consistent across bands, no? 
    g[band+'_specs'] = fx.chip_specs(band)
    
    # retrieve chips
    g[band+'_chips'] = av.sort([i for i in fx.chips(band)])
    
    # list of observation dates for this bands chips
    g[band+'_dates'] = av.dates(g[band+'_chips'])

In [5]:
# dates shared across all the bands

def dtstr_to_ordinal(dtstr, iso=True):
    """ Return ordinal from string formatted date"""
    _fmt = '%Y-%m-%dT%H:%M:%SZ' if iso else '%Y-%m-%d %H:%M:%S'
    _dt = datetime.strptime(dtstr, _fmt)
    return _dt.toordinal()



shared_dates = av.intersection([g[i+'_dates'] for i in bands])
shared_dates_ord = tuple([dtstr_to_ordinal(i) for i in shared_dates])

In [6]:
for band in bands:
    # only want data for shared dates
    g[band+"_trimmed"] = av.trim(g[band+"_chips"], shared_dates)
    
    # chip data as numpy array
    g[band+"_numpy"] = [i for i in av.to_numpy(g[band+"_trimmed"], av.byubid(g[band+"_specs"]))]
    
    # band rods
    g[band+"_rods"] = av.rods(g[band+"_numpy"])
    
    # numpy array of coordinates (lon, lat) for each pixel 
    g[band+"_locs"] = chip.locations(g[band+"_specs"][0]['chip_x'], 
                                     g[band+"_specs"][0]['chip_y'], 
                                     g[band+'_specs'][0])
    
    # tie our rods to the correct lon and lat
    g[band+"_locrods"] = av.locrods(g[band+"_locs"], g[band+"_rods"])

In [7]:
# we now have a dictionary per band, keys being the lon/lat for a rods worth of band values, sorted by date
blue_locrods[(4920.0, -5850.0)]

array([-9999,  4513,  5522, ...,  1191,  4702,   685], dtype=int16)

In [8]:
for i in blue_specs:
    print(i['chip_x'], i['chip_y'])
    

3000 -3000
3000 -3000
3000 -3000
3000 -3000


In [9]:
# chip x and y
# picking on blue
chip_x, chip_y = blue_specs[0]['chip_x'], blue_specs[0]['chip_y']

In [10]:
print(len(blue_locs))
print(len(blue_locs[0]))

100
100


In [11]:
len(blue_locs)

100

In [12]:
pixel_coords = np.reshape(blue_locs, (10000, 2))

In [13]:
blue_locrods[tuple(pixel_coords[0])]

array([8452, 3687, 6506, ..., 1400, 4912,  612], dtype=int16)

In [14]:
# what we need is a list of tuples:  [((pixel coordinate pair), {dict of band rods})]
rdd_able = []
for coords in pixel_coords:
    _d = dict()
    for band in bands:
        _d[band] = g[band+"_locrods"][tuple(coords)]
    #_d['dates'] = np.asarray(shared_dates)
    _d['dates'] = shared_dates_ord
    _d['chip_x'] = chip_x
    _d['chip_y'] = chip_y
    _t = (tuple(coords), _d)
    rdd_able.append(_t)

In [15]:
len(rdd_able)

10000

In [16]:
conf = (SparkConf().setAppName("foo").setMaster(LPW_MESOS_MASTER)\
        .set("spark.mesos.executor.docker.image", LPW_EXECUTOR_IMAGE)\
        .set("spark.executor.cores", LPW_EXECUTOR_CORES)\
        .set("spark.mesos.executor.docker.forcePullImage", LPW_EXECUTOR_FORCE_PULL_IMAGE))

In [17]:
sc = SparkContext(conf=conf)

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.UnsatisfiedLinkError: /usr/lib/libmesos-1.1.0.so: libsvn_delta-1.so.1: cannot open shared object file: No such file or directory
	at java.lang.ClassLoader$NativeLibrary.load(Native Method)
	at java.lang.ClassLoader.loadLibrary0(ClassLoader.java:1941)
	at java.lang.ClassLoader.loadLibrary(ClassLoader.java:1857)
	at java.lang.Runtime.loadLibrary0(Runtime.java:870)
	at java.lang.System.loadLibrary(System.java:1122)
	at org.apache.mesos.MesosNativeLibrary.load(MesosNativeLibrary.java:54)
	at org.apache.mesos.MesosNativeLibrary.load(MesosNativeLibrary.java:79)
	at org.apache.spark.SparkContext$.org$apache$spark$SparkContext$$createTaskScheduler(SparkContext.scala:2487)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:489)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)

In [49]:
short_rdd_able = rdd_able[:5]

In [50]:
len(short_rdd_able)

5

In [51]:
short_rdd = sc.parallelize(short_rdd_able, 5)

In [52]:
short_rdd_able[0][1]['dates']

(734245,
 730254,
 732638,
 730349,
 734286,
 732078,
 725854,
 729573,
 724725,
 726366,
 730381,
 733278,
 727381,
 731381,
 730718,
 732150,
 732126,
 728789,
 725310,
 730717,
 728430,
 733525,
 731086,
 724462,
 730661,
 731238,
 731366,
 730654,
 735373,
 731222,
 731149,
 732182,
 729550,
 729717,
 725934,
 732950,
 733318,
 731301,
 727797,
 733005,
 729646,
 733613,
 733302,
 732285,
 730262,
 730813,
 728437,
 732046,
 729278,
 732421,
 729301,
 731653,
 729678,
 733493,
 730758,
 732237,
 726350,
 725646,
 731310,
 732862,
 731869,
 728629,
 728174,
 729958,
 733254,
 733117,
 727102,
 728533,
 730070,
 726734,
 735286,
 733501,
 731142,
 732006,
 734717,
 731933,
 732518,
 725205,
 731070,
 730974,
 728773,
 734437,
 733382,
 724446,
 733245,
 729790,
 729861,
 733150,
 727278,
 725166,
 731677,
 734269,
 729429,
 732982,
 733590,
 730486,
 732094,
 733630,
 728590,
 727653,
 734678,
 733078,
 728229,
 734189,
 730662,
 734918,
 731662,
 735942,
 732509,
 731861,
 730278,
 

In [21]:
import ccd

In [56]:
def run_detect(indata):
    _coords = indata[0]
    _chipy = indata[1]['chip_y']
    _chipx = indata[1]['chip_x']
    _results = ccd.detect(dates=indata[1]['dates'], 
                          blues=indata[1]['blue'], 
                          greens=indata[1]['green'], 
                          reds=indata[1]['red'], 
                          nirs=indata[1]['nir'],
                          swir1s=indata[1]['swir1'], 
                          swir2s=indata[1]['swir1'], 
                          thermals=indata[1]['thermal'], 
                          quality=indata[1]['cfmask'])
    return _coords, _chipx, _chipy, _results

In [57]:
short_map = short_rdd.map(lambda i: run_detect(i))

In [58]:
short_results = short_map.collect()

In [59]:
short_results[3]

((3000.0, -3090.0),
 3000,
 -3000,
 {'algorithm': 'lcmap-pyccd:1.3.0',
  'change_models': [ChangeModel(start_day=724414, end_day=735853, break_day=735853, observation_count=750, change_probability=0, curve_qa=8, blue=SpectralModel(rmse=241.03845031482103, coefficients=array([ -1.40216517e-03,   8.85319707e+00,   1.50087446e+00,
         0.00000000e+00,   0.00000000e+00,  -0.00000000e+00,
         3.66849269e+00]), intercept=2031.5351248291481, magnitude=223.63950677524031), green=SpectralModel(rmse=277.0962462054087, coefficients=array([ -4.66296453e-04,   9.91302596e+00,   1.02880706e+00,
         0.00000000e+00,   0.00000000e+00,   1.27255683e+00,
         7.84822437e+00]), intercept=1668.2816488714091, magnitude=287.30251077860862), red=SpectralModel(rmse=304.06046438950665, coefficients=array([  6.57351475e-05,   1.16893573e+01,  -1.19117279e+00,
         0.00000000e+00,   0.00000000e+00,  -6.58339804e-01,
         1.22882610e+01]), intercept=1533.0139618233061, magnitude=363.75175

In [17]:
# 10 vs 100 to avoid overwhelming the notebook
for x in range(0, 10):
    for y in range(0, 10):
        print("x: {}, y: {}".format(x, y))
        _coords = tuple(blue_locs[x][y]) 
        print(_coords)
        print(blue_locrods[_coords])
        # wrapper for ccd.detect to be run by spark
        # run_detect(_coords, chip_x, chip_y, 
        #            blue_assoc[_coords],
        #            cfmask_assoc[_coords],
        #            green_assoc[_coords],
        #            nir_assoc[_coords],
        #            red_assoc[_coords],
        #            swir1_assoc[_coords],
        #            swir2_assoc[_coords],
        #            thermal_assoc[_coords])
        

x: 0, y: 0
(3000.0, -3000.0)
[8452 3687 6506 ..., 1400 4912  612]
x: 0, y: 1
(3000.0, -3030.0)
[8380 3887 6472 ..., 1353 4760  618]
x: 0, y: 2
(3000.0, -3060.0)
[8525 4091 6622 ..., 1343 4839  638]
x: 0, y: 3
(3000.0, -3090.0)
[8461 4103 6448 ..., 1369 4973  671]
x: 0, y: 4
(3000.0, -3120.0)
[ 8518 -9999  6212 ...,  1395  4921   699]
x: 0, y: 5
(3000.0, -3150.0)
[ 8458 -9999  6189 ...,  1563  4813   664]
x: 0, y: 6
(3000.0, -3180.0)
[ 8551 -9999  6281 ...,  1528  4852   722]
x: 0, y: 7
(3000.0, -3210.0)
[ 8560 -9999  6355 ...,  1541  4911   747]
x: 0, y: 8
(3000.0, -3240.0)
[-9999 -9999  6281 ...,  1609  4920   686]
x: 0, y: 9
(3000.0, -3270.0)
[-9999 -9999  6076 ...,  1659  4962   689]
x: 1, y: 0
(3030.0, -3000.0)
[8459 3678 6587 ..., 1458 4797  637]
x: 1, y: 1
(3030.0, -3030.0)
[8413 3758 6451 ..., 1365 4759  611]
x: 1, y: 2
(3030.0, -3060.0)
[8511 3912 6386 ..., 1399 4863  574]
x: 1, y: 3
(3030.0, -3090.0)
[8461 3997 6322 ..., 1423 4921  650]
x: 1, y: 4
(3030.0, -3120.0)
[ 8458 -999